# Point Time Series extraction

## 0 - initialisation steps

### 0.1 - Import libs

In [ ]:
import time 
from pathlib import Path

import ee

import helpers as h

# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

### 0.2 - Basic inputs

In [ ]:
country = "kenya" # country case

grid_size = 0.5 # Grid Size (in degrees)

workers = 10 # number of parallel EE requests

asset = "my_featurecollection_path" # the point featurecollection asset
point_id_name = "Point_ID"  # the column/property within your point feature collection from which to take the UNIQUE point id

# Time of interest (onsidering the historical period)
start_date = "2010-01-01"
end_date = "2018-12-31"

# Monitoring period
start_monitor = "2014-01-01"

# Directory where output and temp files will go
# relative path from home
outdir = f"ts_extract/{country.lower()}/"

# Select algorithms to run
cusum_deforest = True
bfast_monitor = True
bs_slope = True
ts_metrics = True
ccdc = True

### DO NOT CHANGE YET ###
# bandname
band='ndvi'
satellite='Landsat'

### 0.3 - Generate the aoi  

In [ ]:
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', country.capitalize())) # here any feature collection can be selected
fc = ee.FeatureCollection(asset).filterBounds(aoi)

## 1 - extract time series

### 1.1 - Algorithm parameter settings

In [ ]:
# bfast parameters
bfast_params = {
    'run': bfast_monitor,
    'start_monitor': start_monitor, 
    'freq': 365,
    'k': 3, 
    'hfrac':0.25, 
    'trend': False, 
    'level':0.05, 
    'backend':'python'}

cusum_params = {
    'run': cusum_deforest,
    'nr_of_bootstraps': 1000
}

bs_slope_params = {
    'run': bs_slope,
    'nr_of_bootstraps': 1000
}

ts_metrics_params = {
    'run': ts_metrics,
    'outlier_removal': True,
    'z_threshhold': 3
}

ccdc_params = {
    'run': ccdc,
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir': str(Path.home()/outdir),
    'workers': workers,
    'ts_params': {
        'start_date': start_date,
        'start_monitor': start_monitor,
        'end_date': end_date,
        'point_id': point_id_name,
        'grid_size': grid_size,
        'band': band,
        'satellite': satellite
    },
    'bfast_params': bfast_params,
    'cusum_params': cusum_params,
    'bs_slope_params': bs_slope_params,
    'ts_metrics_params': ts_metrics_params,
    'ccdc_params': ccdc_params
}

### 1.2 - Run the whole thing

In [ ]:
h.get_change_data(aoi, fc, config_dict)